Libraries

In [9]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.metrics import accuracy_score
from scipy.io import loadmat

Implement function C

In [10]:
def kmeanscluster(X, k, mu, tol=1e-4, maxIter=100):
    n, d = X.shape
    C = np.zeros(n)  # Cluster assignments
    for _ in range(maxIter):
        prev_mu = mu.copy()
        # Assign each data point to the nearest cluster
        for i in range(n):
            distances = np.linalg.norm(X[i] - mu, axis=1)
            C[i] = np.argmin(distances)
        # Update cluster centers
        for j in range(k):
            mu[j] = np.mean(X[C == j], axis=0)
        # Check for convergence
        if np.linalg.norm(mu - prev_mu) < tol:
            break
    return C, mu

این تابع یک الگوریتم خوشه بندی پیاده سازی میکند که ورودی های مطابق آنچه در صورت سوال توضیح داده شد هستند.ابتدا هر نقطه را به نزدیکترین مرکز خوشه نسبت میدهد و خوشه متناظر آنرا به آن نقطه اختصاص میدهد. سپس مراکز خوشه هارا به روز رسانی میکند. به این صورت که مرکز هر خوشه را برابر با میانگین داده های آن خوشه قرار میدهد.و در انتها بررسی میکند که مراکز خوشه ها تغیر کرده اند یا خیر.اگر تغیرات کمتر از مقدار تعیین شده باشد الگوریتم متوقف میشود و نتیجه نهایی خوشه بندی و مرکز خوشه هارا برمیگرداند

Load dataset

In [11]:
features_df = pd.read_csv("breast_data.csv")
X = features_df.values
target_df = pd.read_csv("breast_labels.csv")
y = target_df.values.flatten()

در این بخش دیتاست را لود کرده ایم و فیچر ها و تارگت را جدا کرده ایم . در این مساله تارگت به صورت باینری است پس در مساله طبقه بندی هستیم

Standardize the features

In [12]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

برای نتیجه بهتر فیچر هارا استاندارد کرده ایم که این کار به کمک دو فرمول امکان پذیر است که ما از یکی از آنها به دلخواه در اینجا استفاده کرده ایم

K_means

In [13]:
k=2 #number of clusters
maxIter=100

# Fixing the starting point outside the loop
mu = X_scaled[np.random.choice(X_scaled.shape[0], k, replace=False)]  

# Perform k-means clustering
C, centroids = kmeanscluster(X_scaled, k, mu, tol=1e-4, maxIter=maxIter)

در اینجا چون در مساله طبقه بندی باینری هستیم تعداد خوشه هارا برابر ۲ گذاشتیم . تعداد تکرار هاراهم به صورت تجربی برابر ۱۰۰ قرار دادیم. طبق بررسی های دستی من مقدار تکرار تاثیر زیادی بر روی دقت نداشت. سپس، مراکز اولیه خوشه‌ها به صورت تصادفی از بین  نقاط داده‌های استاندارد‌شده انتخاب می‌شوند و  تا پایان اجرای الگوریتم ثابت می‌مانند. این مراکز اولیه با استفاده از تابع نامپای رندوم از بین نقاط داده‌های استاندارد‌شده انتخاب می‌شوند 

Accuracy

In [14]:
accuracy1 = accuracy_score(y, C)
accuracy2 = accuracy_score(y, 1 - C)
# Choose the better accuracy
accuracy = max(accuracy1, accuracy2)
print(accuracy,silhouette_score(X,y))
print("Accuracy of k-means clustering:", accuracy)

0.9066901408450704 0.5135336980666892
Accuracy of k-means clustering: 0.9066901408450704


علت تعریف دقت به این صورت این اتفاق بود که گاهی دقت مدل ۹۱ درصد و گاهی ۹ درصد اعلام میشد. علت این اتفاق این بود که چون در مساله طبقه بندی باینری هستیم انگار لیبل ها جابه جا میشد. بنابراین دقت نهایی را ماکسیمم دو دقت در نظر گرفتیم.

5 different starting points

In [15]:
k = 2  # Number of clusters
maxIter = 100
num_starting_points = 5

max_accuracy = 0
best_C = None
best_centroids = None

for i in range(num_starting_points):
    # Fixing the starting point outside the loop
    mu = X_scaled[np.random.choice(X_scaled.shape[0], k, replace=False)]
    # Perform k-means clustering
    C, centroids = kmeanscluster(X_scaled, k, mu, tol=1e-4, maxIter=maxIter)
    # Calculate accuracy of clustering
    accuracy1 = accuracy_score(y, C)
    accuracy2 = accuracy_score(y, 1 - C)
    # Choose the better accuracy
    accuracy = max(accuracy1, accuracy2)
    print("-------------------")
    print("starting point :",i)
    print(accuracy,silhouette_score(X,y))
    

-------------------
starting point : 0
0.9049295774647887 0.5135336980666892
-------------------
starting point : 1
0.9119718309859155 0.5135336980666892
-------------------
starting point : 2
0.9119718309859155 0.5135336980666892
-------------------
starting point : 3
0.9049295774647887 0.5135336980666892
-------------------
starting point : 4
0.9119718309859155 0.5135336980666892


در این بخش الگوریتم قبلی را برای ۵ نقطه شروع متمایز که به صورت رندوم انتخاب شده اند ، بررسی کردیم. که این کار به کمک یک حلقه تکرار انجام شده است. همانظور که مشاهده میشود دقت های بدست آمده خیلی با یکدیگر تفاوتی ندارد و این نشان دهنده این است که الگوریتممان خیلی وابسته به نقطه شروع ای که تصادفی انتخابش میکنیم نیست

problem 4-question b- answer:

My Observations:
Running the k-means clustering algorithm multiple times with different starting points can lead to different clustering results.
The accuracy of the clustering may vary depending on the starting points chosen.
In some cases, the algorithm may converge to a suboptimal solution, resulting in lower accuracy.
However, in general, the accuracy tends to be relatively consistent across different runs, especially if the dataset is well-separated and the clusters are distinct.
Choosing an appropriate number of clusters (k) and optimizing other parameters such as tolerance and maximum iterations can also impact the clustering accuracy.

 using provided initial centres

In [24]:
# Load initial centers from init_mu.mat
init_mu_data = loadmat("init_mu.mat")

# Inspect the keys in the loaded data
print(init_mu_data.keys())
init_mu = init_mu_data["mu_init"]

k = init_mu.shape[1]  # Number of clusters
maxIter = 100

# Perform k-means clustering using the provided initial centers
C, centroids = kmeanscluster(X_scaled, k, init_mu.T, tol=1e-4, maxIter=maxIter)

# Calculate accuracy of clustering
accuracy1 = accuracy_score(y, C)
accuracy2 = accuracy_score(y, 1 - C)

# Choose the better accuracy
accuracy = max(accuracy1, accuracy2)

# Print the accuracy
print("Accuracy of k-means clustering using provided initial centers:", accuracy)

dict_keys(['__header__', '__version__', '__globals__', 'mu_init'])
Accuracy of k-means clustering using provided initial centers: 0.9066901408450704


این قطعه کد اقدام به خوشه‌بندی روی داده‌ها، با استفاده از مراکز خوشه‌های اولیه که در فایل متلب ارائه شده‌اند، میکند. این توضیح کاملتر این الگوریتم به صورت زیر است:
در ایتدا مراکز اولیه را بارگذاری میکنیم. مراکز اولیه خوشه‌ها از یک فایل متلب با استفاده از تابع لودمت از کتابخانه سایپای بارگذاری می‌شوند. 
سپس داده های بارگذاری شده را بررسی میکنیم.  برای این کار کلیدهای داده‌های بارگذاری شده را چاپ کردم تا ساختار آنها را درک کنم.
سپس مراکز اولیه را استخراج کردم و آنهارا در یک متغیر مطابق آنچا در کد پیاده سازی شده، ذخیره کردم 
 تعداد خوشه ها از شکل آرایه بدست آمده تعیین می‌شود. این کار با استخراج تعداد ستون‌ها، با فرض اینکه هر ستون یک مرکز خوشه را نمایش می‌دهد، صورت می‌گیرد
تعداد حداکثر تکرارها برای الگوریتم مانند قبل روی ۱۰۰ تکرار تنظیم شده است.
به طور کلی، این قطعه کد مراکز اولیه خوشه‌بندی از یک فایل بارگذاری میکند، خوشه‌بندی را با استفاده از این مراکز اولیه انجام می‌دهد، سپس دقت خوشه‌بندی را محاسبه و چاپ می‌کند 


----------------------------------------------------------------------------------------
problem 4 - question d- answer

What happens if you initialize with the true centres, obtained after the true clustering?

Perfect Clustering: If you initialize the centroids with the true centers obtained from the true clustering, the k-means algorithm might converge quickly to a solution where each data point is assigned to its correct cluster. This could result in high accuracy when evaluating the clustering against the true labels.

Overestimation of Algorithm Performance: Using the true centers for initialization might artificially inflate the perceived performance of the k-means algorithm. It may not reflect the algorithm's actual performance on unseen data or in real-world scenarios where true centers are unknown.

Assumption Violation: Initializing with true centers assumes perfect knowledge of the data distribution, which is usually not available in real-world scenarios. This violates the assumptions of unsupervised learning, where the algorithm should learn patterns and structures from the data itself rather than relying on external information.

Robustness Testing: While initializing with true centers can be useful for testing the robustness of the algorithm or as a sanity check, it's essential to evaluate the algorithm's performance under more realistic conditions, such as using random initialization or other initialization methods commonly employed in k-means.

In summary, initializing with true centers can lead to overfitting and biased evaluation of the algorithm's performance. It's more informative to evaluate the algorithm under typical conditions, such as using random initialization, to assess its generalization ability and robustness.

















----------------------------------------------------------------------------------------
problem 4- question e- answer :


Can you achieve better accuracy using another unsupervised learning method? What about
a supervised one? Explain. 

To potentially achieve better accuracy using another unsupervised learning method or a supervised one, we need to consider the characteristics of the data, the nature of the problem, and the strengths and weaknesses of different algorithms. Here's how each scenario might unfold:

Using Another Unsupervised Learning Method:

Hierarchical Clustering: This method does not require specifying the number of clusters in advance and can capture hierarchical relationships between clusters. If the data exhibits hierarchical structure or varying cluster densities, hierarchical clustering might yield better results than k-means.
DBSCAN (Density-Based Spatial Clustering of Applications with Noise): DBSCAN can identify clusters of arbitrary shapes and sizes and is robust to noise and outliers. If the clusters in the data have irregular shapes or varying densities, DBSCAN might outperform k-means.
Gaussian Mixture Models (GMM): GMM assumes that the data is generated from a mixture of several Gaussian distributions. If the data is not well-separated or contains overlapping clusters, GMM might provide better results than k-means.
Using a Supervised Learning Method:

Classification Algorithms (e.g., Support Vector Machines, Random Forests, Neural Networks): By treating clustering as a classification problem and using the true labels for training, supervised learning algorithms can learn complex decision boundaries that might better separate the clusters in the data. However, this approach assumes the availability of labeled data for training, which might not always be feasible or practical.
Semi-supervised Learning: In scenarios where only a subset of the data is labeled, semi-supervised learning methods can leverage both labeled and unlabeled data to improve clustering accuracy. Techniques such as self-training or co-training can be applied to iteratively refine the clustering results using the available labeled data.
In summary, achieving better accuracy using another unsupervised learning method or a supervised one depends on various factors such as the data characteristics, problem complexity, availability of labeled data, and algorithm suitability. It's essential to consider these factors and experiment with different algorithms to determine the most appropriate approach for your specific task.